In [ ]:
%load_ext autoreload
%autoreload 2
# %matplotlib widget

import warnings
warnings.simplefilter('error', UserWarning)
import os,sys
import h5py  
import numpy as np
import math,cmath
import pickle
from scipy.optimize import curve_fit
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('default')
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['figure.titlesize'] = 20
mpl.rcParams['figure.figsize'] = [6.4*1.2,4.8*1.2]
mpl.rcParams['axes.labelsize'] = 30
mpl.rcParams['axes.titlesize'] = 30
mpl.rcParams['lines.marker'] = 's'
mpl.rcParams['lines.linestyle'] = ''
mpl.rcParams['lines.markersize'] = 12
mpl.rcParams['errorbar.capsize'] = 12
mpl.rcParams['xtick.labelsize'] = mpl.rcParams['ytick.labelsize'] = 22
mpl.rcParams['legend.fontsize'] = 24
mpl.rcParams.update({"axes.grid" : True})
import util as yu

# yu.flag_fast=True

In [ ]:
path='/p/project/pines/li47/code/projectData/NST_b-discNJN-old/data_merge/NST_b.h5_main'
path='/p/project/pines/li47/code/projectData/NST_b-discNJN-old/data_merge/NST_b.h5_discD'
data=yu.load(path)
for ens in yu.ensembles:
    Ncfg=yu.deepKey(data[ens]['2pt'],2).shape[0]
    print(ens+': Ncfg='+str(Ncfg))

In [ ]:
def getdat(ens,ins,tfs,case):
    flags={
        'cc2pt':False,
        'cc3pt':False
    }
    spins=[0,1]; flas=['N1','N2']; diags=['N','N_bw']
    # spins=[0,1]; flas=['N1']; diags=['N']
    diags_3pt=[diag+'-j' for diag in diags]
    t=np.mean([data[ens]['2pt']['t;0,0,0;{};{}_t;0,0,0;{};{}'.format(spin,fla,spin,fla)][diag] for spin in spins for fla in flas for diag in diags],axis=0)
    if flags['cc2pt']:
        t=(t+np.conj(t))/2
    t_2pta=t_2ptb=t
    
    cg,j=ins.split('_')
    gm={'gS':'id','gA':'g5gz','gT':'sgmxy'}[cg]
    pol={'gS':1,'gA':-1,'gT':-1}[cg]
    factor={'gS':1,'gA':1j,'gT':-1j}[cg]

    t_3pt={}
    for tf in tfs:
        insert='_'.join([gm,j,str(tf)])
        t_conn=data[ens]['3pt']['0mom'][ins+'_'+str(tf)]['NJN'] if 'conn' in case else 0
        fla2sgn={'N1':1,'N2':(-1 if j=='j-' else 1)*{'id':1,'g5gz':1,'sgmxy':1}[gm]}
        t_disc=np.mean([(
            data[ens]['3pt']['t;0,0,0;0;{}_t;0,0,0;0;{}'.format(fla,fla)][insert][diag]+
            data[ens]['3pt']['t;0,0,0;1;{}_t;0,0,0;1;{}'.format(fla,fla)][insert][diag]*pol
        )/2*fla2sgn[fla]*factor
                        for fla in flas for diag in diags_3pt],axis=0)
        t_3pt[tf]={'conn':t_conn,'disc':t_disc,'conn+disc':t_conn+t_disc}[case]
        if flags['cc3pt']:
            t_3pt[tf]= (t_3pt[tf] + np.conj(t_3pt[tf][:,::-1])*yu.gtCj[gm]*(np.conj(factor)/factor))/2
        
        t_VEV=data[ens]['VEV']['j']['_'.join([gm,j])]
    
    t=[[t_3pt[tf] for tf in tfs],[t_2pta,t_2ptb],[t_VEV]]
    return t

In [ ]:
# temp for xcheck with Giannis

def jackknife(dat_ens,d=1):
    n=len(dat_ens)
    return np.array([np.mean(np.delete(dat_ens,i,axis=0),axis=0) for i in range(n)])

path='/p/project/pines/li47/code/glwc2/example/basicLatticeAnalysis/dat/NST_b_cC80_temp.h5'
path2='/p/project/pines/li47/code/projectData/NST_b-discNJN-old/data_merge/NST_b.h5_discD'
with h5py.File(path,'w') as f, h5py.File(path2) as f2:
    tfs=range(6,26+1,2)
    print(f2['cC211.060.80'].keys())
    f.copy(f2['cC211.060.80/cfgs'],f,name='cfgs')
    t=getdat('cC211.060.80','gT_j+',tfs,'disc')
    for ele in t[0]:
        f.create_dataset('gT_j+_{}'.format(ele.shape[-1]-1),data=ele)
    t2pt=t[1][0]
    f.create_dataset('2pt',data=t2pt)
    c3pt_jk_dic={tf:jackknife(t[0][i]) for i,tf in enumerate(tfs)}
    c2pt_jk=jackknife(t2pt)
    ratio_jk_dic={tf:np.real(c3pt_jk_dic[tf])/np.real(c2pt_jk)[:,tf][:,None] for tf in tfs}
    for tf in tfs:
        f.create_dataset('ratio_j+_{}_jk'.format(tf),data=ratio_jk_dic[tf])

In [ ]:
# tfmax check
thred=0.2

ens2tfmax={}
for irow,ens in enumerate(yu.ensembles):
    ins='gS_j+'
    # tfs=[int(insert.split('_')[-1]) for insert in data[ens]['3pt']['0mom'].keys() if insert.startswith(ins)]; tfs.sort()
    dat=getdat(ens,ins,[10],'disc')
    
    def func(dat):
        t=yu.meanDeep(dat)
        return np.real(t[1][0])
    
    (mean,err,cov)=yu.jackknife(dat,func)
    errR=err/np.abs(mean)
    for tf,ele in enumerate(errR[0]):
        if ele>thred:
            ens2tfmax[ens]=tf-1
            break
print(ens2tfmax)

# for ins in ['gS_j+','gS_j-','gA_j+','gA_j-','gT_j+','gT_j-']:
# for ins in ['gS_j-']:
#     print()
#     for iens,ens in enumerate(yu.ensembles):
#         for icase,case in enumerate(['conn','conn+disc','disc']):
#             tfs=[int(insert.split('_')[-1]) for insert in data[ens]['3pt']['0mom'].keys() if insert.startswith(ins)]; tfs.sort()
#             dat=getdat(ens,ins,tfs,'conn')
#             needsVEV=True if case!='conn' and ins in ['gS_j+'] else False
#             def func(dat):
#                 t=yu.meanDeep(dat)
#                 t=[np.real((t[0][i]- (t[1][0][tf]*t[2][0] if needsVEV else 0))) for i,tf in enumerate(tfs) ]
#                 return t
#             (mean,err,cov)=yu.jackknife(dat,func)
#             errMax=0
#             for i_tf,tf in enumerate(tfs):
#                 errR=err[i_tf]/np.abs(mean[i_tf])
#                 errMax=np.max([errMax,np.max(errR)])
#                 for i_tc,tc in enumerate(range(0,tf+1,1)):
#                     if tc in [0,tf]:
#                         continue
#                     if errR[i_tc]>0.3:
#                         print(ins,ens,case,tf,tc,errR[i_tc])
            # print(ins,ens,case,errMax)


In [ ]:
func_C2pt_1st=lambda t,E0,c0: c0*np.exp(-E0*t)
func_C2pt_2st=lambda t,E0,c0,dE1,rc1: c0*np.exp(-E0*t)*(1 + rc1*np.exp(-dE1*t))
func_C2pt_3st=lambda t,E0,c0,dE1,rc1,dE2,rc2: c0*np.exp(-E0*t)*(1 + rc1*np.exp(-dE1*t) + rc2*np.exp(-dE2*t))

ens2info={
    'cB211.072.64':
        {
            'tmins_1st':range(8,24+1),
            'tmins_2st':range(1,13+1),
            'tmins_3st':range(1,6+1),
            'xlim2':[0,24],
            'xlim3':[0,13],
            'ylim1':[0.23,0.6],
            'ylim2':[0.35,0.4],
            'ylim3':[0.35,1.5],
            },
    'cC211.060.80':
        {
            'tmins_1st':range(8,29+1),
            'tmins_2st':range(1,18+1),
            'tmins_3st':range(1,8+1),
            'xlim2':[0,29],
            'xlim3':[0,18],
            'ylim1':[0.2,0.5],
            'ylim2':[0.3,0.34],
            'ylim3':[0.3,1.2],
            },
    'cD211.054.96':
        {
            'tmins_1st':range(8,34+1),
            'tmins_2st':range(1,21+1),
            'tmins_3st':range(1,7+1),
            'xlim2':[0,34],
            'xlim3':[0,22],
            'ylim1':[0.2,0.5],
            'ylim2':[0.25,0.3],
            'ylim3':[0.25,1],
            },
}

def run(ens):
    fig, axd = plt.subplot_mosaic([['f1','f1','f1'],['f2','f2','f3']],figsize=(20,8))
    fig.suptitle(ens)
    (ax1,ax2,ax3)=(axd[key] for key in ['f1','f2','f3'])
    ax1.set_xticks(range(0,ens2tfmax[ens]+1,2))  
    ax2.set_xticks(range(0,ens2tfmax[ens]+1,2)) 
    ax3.set_xticks(range(0,ens2tfmax[ens]+1,2)) 
    ax1.set_ylim(ens2info[ens]['ylim1'])
    ax2.set_ylim(ens2info[ens]['ylim2'])
    ax3.set_ylim(ens2info[ens]['ylim3'])
    
    
    tmin=1; tmax=ens2tfmax[ens]
    dat=getdat(ens,'gS_j+',[10],'disc')[1][0]
    def func(dat):
        t=yu.meanDeep(dat)
        t=np.log(t/np.roll(t,-1))
        return np.real(t)
    mean,err,cov=yu.jackknife(dat,func)
    ax1.errorbar(range(tmin,tmax),mean[0][tmin:tmax],yerr=err[0][tmin:tmax],color='black')
    
    # initial pars
    pars0_initial=[0.4,1e-8,0.5,2,0.8,1]
    
    # 1st fit
    pars0=pars0_initial[:2]
    fits=[]
    for tmin_fit in ens2info[ens]['tmins_1st']:
        tfs=np.arange(tmin_fit,tmax+1)
        def func(dat):
            dat=dat[:,tfs]
            t=yu.meanDeep(dat)
            return np.real(t)
        def fitfunc(E0,c0):
            return func_C2pt_1st(tfs,E0,c0)
        pars_mean,pars_err,pars_cov,chi2R_mean,chi2R_err,Ndof=yu.fit(dat,func,fitfunc,pars0=pars0)
        pars0=pars_mean
        # print(tmin_fit,pars_mean)
        ax2.errorbar([tmin_fit],pars_mean[0],yerr=pars_err[0],color='r',mfc='white')
        ax2.annotate("%0.1f" %chi2R_mean,(tmin_fit,pars_mean[0]-pars_err[0]-0.005),color='r')
        fits.append((pars_mean,pars_err,chi2R_mean,Ndof))
    pars_mean_MA,pars_err_MA,probs=yu.modelAvg(fits)
    t_mean=pars_mean_MA[0]; t_err=pars_err_MA[0]
    ax2.fill_between([ens2info[ens]['xlim2'][0]+4,ens2info[ens]['xlim2'][-1]],t_mean-t_err,t_mean+t_err,color='r',alpha=0.2)
    print(ens,'1st',ens2info[ens]['tmins_1st'][probs.argmax()],pars_mean_MA,end='\t')
    
    # 2st fit
    pars0=pars0_initial[:4]
    fits=[]
    for tmin_fit in ens2info[ens]['tmins_2st']:
        tfs=np.arange(tmin_fit,tmax+1)
        def func(dat):
            dat=dat[:,tfs]
            t=yu.meanDeep(dat)
            return np.real(t)
        def fitfunc(E0,c0,E1,rc1):
            return func_C2pt_2st(tfs,E0,c0,E1-E0,rc1)
        pars_mean,pars_err,pars_cov,chi2R_mean,chi2R_err,Ndof=yu.fit(dat,func,fitfunc,pars0=pars0)
        pars0=pars_mean
        # print(tmin_fit,pars_mean[0],pars_err[0])
        ax2.errorbar([tmin_fit],pars_mean[0],yerr=pars_err[0],color='g',mfc='white')
        ax2.annotate("%0.1f" %chi2R_mean,(tmin_fit,pars_mean[0]-pars_err[0]-0.005),color='g')
        ax3.errorbar([tmin_fit],pars_mean[2],yerr=pars_err[2],color='g',mfc='white')
        ax3.annotate("%0.1f" %chi2R_mean,(tmin_fit,pars_mean[2]-pars_err[2]-0.005),color='g')
        fits.append((pars_mean,pars_err,chi2R_mean,Ndof))
    pars_mean_MA,pars_err_MA,probs=yu.modelAvg(fits)
    t_mean=pars_mean_MA[0]; t_err=pars_err_MA[0]
    ax2.fill_between([ens2info[ens]['xlim2'][0]+2,ens2info[ens]['xlim2'][-1]-2],t_mean-t_err,t_mean+t_err,color='g',alpha=0.2)
    t_mean=pars_mean_MA[2]; t_err=pars_err_MA[2]
    ax3.fill_between([ens2info[ens]['xlim3'][0]+2,ens2info[ens]['xlim3'][-1]-2],t_mean-t_err,t_mean+t_err,color='g',alpha=0.2)
    
    
    print('2st',ens2info[ens]['tmins_2st'][probs.argmax()],pars_mean_MA,pars_err_MA,end='\t')
    aInv=3.4662684469430785
    print(pars_mean_MA[0]*aInv,pars_err_MA[0]*aInv)
        
    # 3st fit
    pars0=pars0_initial
    fits=[]
    for tmin_fit in ens2info[ens]['tmins_3st']:
        tfs=np.arange(tmin_fit,tmax+1)
        def func(dat):
            dat=dat[:,tfs]
            t=yu.meanDeep(dat)
            return np.real(t)
        def fitfunc(E0,c0,E1,rc1,E2,rc2):
            return func_C2pt_3st(tfs,E0,c0,E1-E0,rc1,E2-E0,rc2)
        pars_mean,pars_err,pars_cov,chi2R_mean,chi2R_err,Ndof=yu.fit(dat,func,fitfunc,pars0=pars0)
        pars0=pars_mean
        # print(tmin_fit,pars_mean)
        ax2.errorbar([tmin_fit],pars_mean[0],yerr=pars_err[0],color='b',mfc='white')
        ax2.annotate("%0.1f" %chi2R_mean,(tmin_fit,pars_mean[0]-pars_err[0]-0.005),color='b')
        ax3.errorbar([tmin_fit],pars_mean[2],yerr=pars_err[2],color='b',mfc='white')
        ax3.annotate("%0.1f" %chi2R_mean,(tmin_fit,pars_mean[2]-pars_err[2]-0.005),color='b')
        fits.append((pars_mean,pars_err,chi2R_mean,Ndof))
    pars_mean_MA,pars_err_MA,probs=yu.modelAvg(fits)
    t_mean=pars_mean_MA[0]; t_err=pars_err_MA[0]
    ax2.fill_between([ens2info[ens]['xlim2'][0],ens2info[ens]['xlim2'][-1]-4],t_mean-t_err,t_mean+t_err,color='b',alpha=0.2)
    t_mean=pars_mean_MA[2]; t_err=pars_err_MA[2]
    ax3.fill_between([ens2info[ens]['xlim3'][0],ens2info[ens]['xlim3'][-1]-4],t_mean-t_err,t_mean+t_err,color='b',alpha=0.2)
    print('3st',ens2info[ens]['tmins_2st'][probs.argmax()],pars_mean_MA)

# for ens in yu.ensembles:
#     run(ens); plt.savefig('fig/2pt_'+ens+'.pdf'); plt.close()

for ens in yu.ensembles[-1:]:
    run(ens); plt.savefig('fig/2pt_'+ens+'.pdf'); plt.close()

cB211.072.64 1st 16 [3.81159042e-01 1.66858071e-08]	2st 7 [3.80305345e-01 1.62458675e-08 5.97738157e-01 8.44149000e-01]	3st 4 [3.78028376e-01 1.53166117e-08 5.49897831e-01 7.31081002e-01
 1.15350108e+00 9.50472487e+00]

cC211.060.80 1st 19 [3.2715705e-01 2.0002938e-09]	2st 10 [3.23009445e-01 1.77755504e-09 4.80835211e-01 1.13744059e+00]	3st 4 [3.20077420e-01 1.60698234e-09 4.27155190e-01 6.60357716e-01
 8.35870865e-01 1.31450910e+00]
 
cD211.054.96 1st 23 [2.72604564e-01 3.76905545e-10]	2st 9 [2.72641767e-01 3.73256804e-10 4.64117754e-01 9.62179328e+00]	3st 3 [2.72388509e-01 3.70104965e-10 4.51551034e-01 7.54966326e-01
 9.13310545e-01 1.22687105e+00]


In [ ]:
ens2tfmin_best={'cB211.072.64': 8, 'cC211.060.80': 10, 'cD211.054.96':9}

In [ ]:
# plateau

def run(ins,settings,label='plateau'):
    fig, axs = yu.getFigAxs(len(yu.ensembles)+1,1+1)
    fig.suptitle(ins)
    yu.addRowHeader(axs,[ens[:2]+ens[-2:] for ens in yu.ensembles]+['Extra'])
    yu.addColHeader(axs,['disc',''])
    for icol in [0,1]:
        axs[0,icol].get_shared_x_axes().join(axs[0,icol],*axs[1:len(yu.ensembles),icol])
    
    fmts=['s','s','s']
    
    func_C2pt_2st=lambda t,E0,c0,dE1,rc1: c0*np.exp(-E0*t)*(1 + rc1*np.exp(-dE1*t))
    func_C3pt_2st=lambda tf,tc,E0,a00,dE1,ra01,ra11: a00*np.exp(-E0*tf)*(1 + ra01*(np.exp(-dE1*tc) + np.exp(-dE1*(tf-tc))) + ra11*np.exp(-dE1*tf))

    
    best={}
    for irow,ens in enumerate(yu.ensembles):
        t={
        'gS_j+':yu.ens2info[ens]['factor_gS'],
        'gS_j-':yu.ens2info[ens]['factor_gS'],
        'gA_j+':yu.ens2info[ens]['factor_gAs'],
        'gA_j-':yu.ens2info[ens]['factor_gAv'],
        'gT_j+':yu.ens2info[ens]['factor_gT'],
        'gT_j-':yu.ens2info[ens]['factor_gT'],
        }
        prefactor=t[ins] if ins in t else 1
        
        xUnit=yu.ens2info[ens]['a']
        
        for icol,case in enumerate(['disc']):
            icol*=2
            ylim=settings['ylim']
            shift_chi2=(ylim[1]-ylim[0])/10
            needsVEV=True if case!='conn' and ins in ['gS_j+'] else False
            tfs=[int(insert.split('_')[-1]) for insert in data[ens]['3pt']['t;0,0,0;0;N1_t;0,0,0;0;N1'].keys() if insert.startswith('id_j+')]; tfs.sort()
            tfs=[tf for tf in tfs if tf<=settings['tfmax_3pt'][irow]]
            
            oneQ = False
            dat=getdat(ens,ins,tfs,case)
            
            # ratio plot
            def func(dat):
                t=yu.meanDeep(dat)
                t=[t[0][i][1:tf]/np.sqrt(
                    np.real(t[1][0][tf])*np.real(t[1][1][tf])*\
                    np.real(t[1][0][1:tf][::-1])/np.real(t[1][0][1:tf])*\
                    np.real(t[1][1][1:tf])/np.real(t[1][1][1:tf][::-1])
                ) - (t[2][0] if needsVEV else 0) 
                for i,tf in enumerate(tfs)]
                # print(t)
                t=yu.prefactorDeep(t,prefactor)
                return t
            (mean,err,cov) = yu.jackknife(dat,func)
            for i_tf,tf in enumerate(tfs):
                tMean=mean[i_tf];tErr=err[i_tf]
                axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)*xUnit,tMean,tErr,fmt=fmts[irow])
                axs[irow,icol].set_ylim(ylim)
            # axs[irow,icol].set_xlim([-12*xUnit,12*xUnit])
                
            fits=[]
            for datmin in settings['datmin'][irow]:
                print(ins,ens,case,datmin,end='                      \r')
                datmin_2pt=datmin_3pt_tf=datmin_3pt_tci=datmin_3pt_tcf=datmin
                datmin_2pt=ens2tfmin_best[ens]
                tfs_3pt_fit=[tf for tf in tfs if tf>=datmin_3pt_tf and tf+1>(datmin_3pt_tci+datmin_3pt_tcf)]
                tcs_3pt_fit={tf:np.arange(datmin_3pt_tci,tf+1-datmin_3pt_tcf) for tf in tfs_3pt_fit}
                
                def func(dat):
                    t=yu.meanDeep(dat)
                    t=[np.array(t[0][i][:tf+1]/np.sqrt(
                            np.real(t[1][0][tf])*np.real(t[1][1][tf])*\
                            np.real(t[1][0][0:tf+1][::-1])/np.real(t[1][0][0:tf+1])*\
                            np.real(t[1][1][0:tf+1])/np.real(t[1][1][0:tf+1][::-1])
                        ) - (t[2][0] if needsVEV else 0))[tcs_3pt_fit[tf]]
                        for i,tf in enumerate(tfs) if tf in tfs_3pt_fit]
                    t=yu.prefactorDeep(t,prefactor)
                    return t
                
                def fitfunc(g):
                    return [[g for tc in tcs_3pt_fit[tf]] for tf in tfs_3pt_fit]
                
                pars_mean,pars_err,pars_cov,chi2R_mean,chi2R_err,Ndof=yu.fit(dat,func,fitfunc,mask_cov=None,pars0=0)
                fits.append((pars_mean,pars_err,chi2R_mean,Ndof))
                axs[irow,icol+1].errorbar([datmin*xUnit],[pars_mean[0]],[pars_err[0]],fmt=fmts[irow],color='b',mfc='white')
                axs[irow,icol+1].annotate("%0.1f" %chi2R_mean,(datmin*xUnit,pars_mean[0]-pars_err[0]-shift_chi2),color='b')
                axs[irow,icol+1].set_ylim(ylim)
            # axs[irow,icol+1].set_xlim([0,10*xUnit])
                
            # modelAvg
            pars_mean_MA,pars_err_MA,probs=yu.modelAvg(fits)            
            t_mean=pars_mean_MA[0]; t_err=pars_err_MA[0]
            best[(ens,case)]=[yu.ens2info[ens]['a']**2,t_mean,t_err]
            axs[irow,icol+1].fill_between([0,(settings['datmin'][irow][-1]+1)*xUnit],t_mean-t_err,t_mean+t_err,color='b',alpha=0.2)
    
    for icol,case in [(1,'disc')]:
        # continue
        irow=len(yu.ensembles)
        a2s=np.array([best[(ens,case)][0] for ens in yu.ensembles])
        means=np.array([best[(ens,case)][1] for ens in yu.ensembles])
        errs=np.array([best[(ens,case)][2] for ens in yu.ensembles])
        axs[irow,icol].errorbar(a2s,means,errs,color='b',mfc='white')
        axs[irow,icol].set_ylim(ylim)
        
        def fitfunc(a2,g0):
            return g0+0*a2
        popt,pcov=curve_fit(fitfunc,a2s,means,sigma=errs,absolute_sigma=True)
        r=fitfunc(a2s,*popt)-means
        # print(r)
        # print(errs)
        chi2R=r.T @ np.linalg.inv(np.diag(errs**2)) @ r / (3-1)
        a2s2=np.arange(0,0.008,0.001)
        res=fitfunc(a2s2,*popt)
        axs[irow,icol].plot(a2s2,res,'r-')
        axs[irow,icol].fill_between(a2s2,res-[np.sqrt(pcov[0,0])]*len(a2s2),res+[np.sqrt(pcov[0,0])]*len(a2s2),color='r',alpha=0.2)
        axs[irow,icol].errorbar([0],[popt[0]],np.sqrt(pcov[0,0]),color='r')
        axs[irow,icol].annotate("%0.1f" %chi2R,(0,popt[0]-np.sqrt(pcov[0,0])-shift_chi2),color='r')
        
        def fitfunc(a2,g0,g1):
            return g0+g1*a2
        popt,pcov=curve_fit(fitfunc,a2s,means,sigma=errs,absolute_sigma=True)
        r=fitfunc(a2s,*popt)-means
        # print(r)
        # print(errs)
        chi2R=r.T @ np.linalg.inv(np.diag(errs**2)) @ r / (3-2)
        a2s2=np.arange(0,0.008,0.001)
        res=fitfunc(a2s2,*popt)
        es=np.array([np.sqrt(np.array([[1,a2]])@pcov@np.array([[1,a2]]).T)[0,0] for a2 in a2s2])
        axs[irow,icol].plot(a2s2,res,'b-')
        axs[irow,icol].fill_between(a2s2,res-es,res+es,color='b',alpha=0.2)
        axs[irow,icol].errorbar([0],[popt[0]],np.sqrt(pcov[0,0]),color='b')
        axs[irow,icol].annotate("%0.1f" %chi2R,(0,popt[0]-np.sqrt(pcov[0,0])-shift_chi2),color='b')

    plt.savefig('fig/'+ins+'_'+label+'.pdf')
    plt.close()
    
settings={
    'datmin':[range(1,9+1),range(1,10+1),range(1,12+1)],
    'tfmax_3pt':[20,22,26],
    'ylim':[-0.8,0.4],
}
run('gS_j-',settings)

settings={
    'datmin':[range(1,9+1),range(1,10+1),range(1,12+1)],
    'tfmax_3pt':[20,22,26],
    'ylim':[-0.03,0.03],
}
run('gA_j-',settings)

# yu.flag_fit_cov2err=True
# settings={
#     'datmin':[range(1,12+1),range(1,12+1),range(1,12+1)],
#     'tfmax_3pt':[26,26,26],
#     'ylim':[-0.03,0.03],
# }
# run('gA_j-',settings,label='plateau_uncorrealted')
# yu.flag_fit_cov2err=False

settings={
    'datmin':[range(1,4+1),range(1,5+1),range(1,6+1)],
    'tfmax_3pt':[10,12,14],
    'ylim':[-0.05,0.05],
}
run('gT_j-',settings)

In [ ]:
# summation

def run(ins,settings):
    fig, axs = yu.getFigAxs(len(yu.ensembles)+1,3)
    fig.suptitle(ins)
    yu.addRowHeader(axs,[ens[:2]+ens[-2:] for ens in yu.ensembles]+['Extra'])
    yu.addColHeader(axs,['disc',''])
    for icol in [0,1]:
        axs[0,icol].get_shared_x_axes().join(axs[0,icol],*axs[1:len(yu.ensembles),icol])
    
    fmts=['s','s','s']
    
    func_C2pt_2st=lambda t,E0,c0,dE1,rc1: c0*np.exp(-E0*t)*(1 + rc1*np.exp(-dE1*t))
    func_C3pt_2st=lambda tf,tc,E0,a00,dE1,ra01,ra11: a00*np.exp(-E0*tf)*(1 + ra01*(np.exp(-dE1*tc) + np.exp(-dE1*(tf-tc))) + ra11*np.exp(-dE1*tf))

    
    best={}
    for irow,ens in enumerate(yu.ensembles):
        t={
        'gS_j+':yu.ens2info[ens]['factor_gS'],
        'gS_j-':yu.ens2info[ens]['factor_gS'],
        'gA_j+':yu.ens2info[ens]['factor_gAs'],
        'gA_j-':yu.ens2info[ens]['factor_gAv'],
        'gT_j+':yu.ens2info[ens]['factor_gT'],
        'gT_j-':yu.ens2info[ens]['factor_gT'],
        }
        prefactor=t[ins] if ins in t else 1
        
        xUnit=yu.ens2info[ens]['a']
        
        for icol,case in enumerate(['disc']):
            icol*=3
            ylim=settings['ylim']
            shift_chi2=(ylim[1]-ylim[0])/10
            needsVEV=True if case!='conn' and ins in ['gS_j+'] else False
            tfs=[int(insert.split('_')[-1]) for insert in data[ens]['3pt']['t;0,0,0;0;N1_t;0,0,0;0;N1'].keys() if insert.startswith('id_j+')]; tfs.sort()
            tfs=np.array([tf for tf in tfs if tf<=settings['tfmax_3pt'][irow]])
            
            oneQ = False
            dat=getdat(ens,ins,tfs,case)
            
            # ratio plot
            def func(dat):
                t=yu.meanDeep(dat)
                t=[t[0][i][1:tf]/np.sqrt(
                    np.real(t[1][0][tf])*np.real(t[1][1][tf])*\
                    np.real(t[1][0][1:tf][::-1])/np.real(t[1][0][1:tf])*\
                    np.real(t[1][1][1:tf])/np.real(t[1][1][1:tf][::-1])
                ) - (t[2][0] if needsVEV else 0) 
                for i,tf in enumerate(tfs)]
                # print(t)
                t=yu.prefactorDeep(t,prefactor)
                return t
            (mean,err,cov) = yu.jackknife(dat,func)
            for i_tf,tf in enumerate(tfs):
                tMean=mean[i_tf];tErr=err[i_tf]
                axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)*xUnit,tMean,tErr,fmt=fmts[irow])
                axs[irow,icol].set_ylim(ylim)
            # axs[irow,icol].set_xlim([-12*xUnit,12*xUnit])
                
            # summation plot
            def func(dat):
                t=yu.meanDeep(dat)
                t=np.real([prefactor*np.sum(t[0][i][1:tf]/np.sqrt(
                    np.real(t[1][0][tf])*np.real(t[1][1][tf])*\
                    np.real(t[1][0][1:tf][::-1])/np.real(t[1][0][1:tf])*\
                    np.real(t[1][1][1:tf])/np.real(t[1][1][1:tf][::-1])
                ) - (t[2][0] if needsVEV else 0))
                for i,tf in enumerate(tfs)])
                return t
            (mean,err,cov) = yu.jackknife(dat,func)
            tMean=mean[0];tErr=err[0]
            axs[irow,icol+1].errorbar(tfs*xUnit,tMean,tErr,fmt=fmts[irow])
            axs[irow,icol+1].axis('auto')
            # axs[irow,icol+1].set_xlim([0,10*xUnit])
            
            fits=[]
            for datmin in settings['datmin'][irow]:
                print(ins,ens,case,datmin,end='                      \r')
                datmin_2pt=datmin_3pt_tf=datmin_3pt_tci=datmin_3pt_tcf=datmin
                datmin_2pt=ens2tfmin_best[ens]
                tfs_3pt_fit=[tf for tf in tfs if tf>=datmin_3pt_tf and tf+1>(datmin_3pt_tci+datmin_3pt_tcf)]
                tcs_3pt_fit={tf:np.arange(datmin_3pt_tci,tf+1-datmin_3pt_tcf) for tf in tfs_3pt_fit}
                
                def func(dat):
                    t=yu.meanDeep(dat)
                    t=np.real([prefactor*np.sum(t[0][i][1:tf]/np.sqrt(
                        np.real(t[1][0][tf])*np.real(t[1][1][tf])*\
                        np.real(t[1][0][1:tf][::-1])/np.real(t[1][0][1:tf])*\
                        np.real(t[1][1][1:tf])/np.real(t[1][1][1:tf][::-1])
                    ) - (t[2][0] if needsVEV else 0))
                    for i,tf in enumerate(tfs) if tf in tfs_3pt_fit])
                    return t
                
                def fitfunc(g,c):
                    return [c+g*tf for tf in tfs_3pt_fit]
                
                pars_mean,pars_err,pars_cov,chi2R_mean,chi2R_err,Ndof=yu.fit(dat,func,fitfunc,mask_cov=None,pars0=[0,0])
                fits.append((pars_mean,pars_err,chi2R_mean,Ndof))
                axs[irow,icol+2].errorbar([datmin*xUnit],[pars_mean[0]],[pars_err[0]],fmt=fmts[irow],color='b',mfc='white')
                axs[irow,icol+2].annotate("%0.1f" %chi2R_mean,(datmin*xUnit,pars_mean[0]-pars_err[0]-shift_chi2),color='b')
                axs[irow,icol+2].set_ylim(ylim)
                
            # modelAvg
            pars_mean_MA,pars_err_MA,probs=yu.modelAvg(fits)            
            t_mean=pars_mean_MA[0]; t_err=pars_err_MA[0]
            best[(ens,case)]=[yu.ens2info[ens]['a']**2,t_mean,t_err]
            axs[irow,icol+2].fill_between([0,(settings['datmin'][irow][-1]+1)*xUnit],t_mean-t_err,t_mean+t_err,color='b',alpha=0.2)
    
    for icol,case in [(2,'disc')]:
        # continue
        irow=len(yu.ensembles)
        a2s=np.array([best[(ens,case)][0] for ens in yu.ensembles])
        means=np.array([best[(ens,case)][1] for ens in yu.ensembles])
        errs=np.array([best[(ens,case)][2] for ens in yu.ensembles])
        axs[irow,icol].errorbar(a2s,means,errs,color='b',mfc='white')
        axs[irow,icol].set_ylim(ylim)
        
        def fitfunc(a2,g0):
            return g0+0*a2
        popt,pcov=curve_fit(fitfunc,a2s,means,sigma=errs,absolute_sigma=True)
        r=fitfunc(a2s,*popt)-means
        # print(r)
        # print(errs)
        chi2R=r.T @ np.linalg.inv(np.diag(errs**2)) @ r / (3-1)
        a2s2=np.arange(0,0.008,0.001)
        res=fitfunc(a2s2,*popt)
        axs[irow,icol].plot(a2s2,res,'r-')
        axs[irow,icol].fill_between(a2s2,res-[np.sqrt(pcov[0,0])]*len(a2s2),res+[np.sqrt(pcov[0,0])]*len(a2s2),color='r',alpha=0.2)
        axs[irow,icol].errorbar([0],[popt[0]],np.sqrt(pcov[0,0]),color='r')
        axs[irow,icol].annotate("%0.1f" %chi2R,(0,popt[0]-np.sqrt(pcov[0,0])-shift_chi2),color='r')
        
        def fitfunc(a2,g0,g1):
            return g0+g1*a2
        popt,pcov=curve_fit(fitfunc,a2s,means,sigma=errs,absolute_sigma=True)
        r=fitfunc(a2s,*popt)-means
        # print(r)
        # print(errs)
        chi2R=r.T @ np.linalg.inv(np.diag(errs**2)) @ r / (3-2)
        a2s2=np.arange(0,0.008,0.001)
        res=fitfunc(a2s2,*popt)
        es=np.array([np.sqrt(np.array([[1,a2]])@pcov@np.array([[1,a2]]).T)[0,0] for a2 in a2s2])
        axs[irow,icol].plot(a2s2,res,'b-')
        axs[irow,icol].fill_between(a2s2,res-es,res+es,color='b',alpha=0.2)
        axs[irow,icol].errorbar([0],[popt[0]],np.sqrt(pcov[0,0]),color='b')
        axs[irow,icol].annotate("%0.1f" %chi2R,(0,popt[0]-np.sqrt(pcov[0,0])-shift_chi2),color='b')

    plt.savefig('fig/'+ins+'_summation.pdf')
    plt.close()
    
settings={
    'datmin':[range(1,7+1),range(1,8+1),range(1,8+1)],
    'tfmax_3pt':[18,20,20],
    'ylim':[-0.8,0.4],
}
run('gS_j-',settings)

settings={
    'datmin':[range(1,8+1),range(1,9+1),range(1,11+1)],
    'tfmax_3pt':[20,24,26],
    'ylim':[-0.03,0.03],
}
run('gA_j-',settings)

# settings={
#     'datmin':[range(1,4+1),range(1,4+1),range(1,4+1)],
#     'tfmax_3pt':[12,12,12],
#     'ylim':[-0.05,0.05],
# }
# run('gT_j-',settings)

In [ ]:
# mid point fit

def run(ins,settings,label='mid'):
    fig, axs = yu.getFigAxs(len(yu.ensembles)+1,3)
    fig.suptitle(ins)
    yu.addRowHeader(axs,[ens[:2]+ens[-2:] for ens in yu.ensembles]+['Extra'])
    yu.addColHeader(axs,['disc',''])
    for icol in [0,1]:
        axs[0,icol].get_shared_x_axes().join(axs[0,icol],*axs[1:len(yu.ensembles),icol])
    
    fmts=['s','s','s']
    
    func_C2pt_2st=lambda t,E0,c0,dE1,rc1: c0*np.exp(-E0*t)*(1 + rc1*np.exp(-dE1*t))
    func_C3pt_2st=lambda tf,tc,E0,a00,dE1,ra01,ra11: a00*np.exp(-E0*tf)*(1 + ra01*(np.exp(-dE1*tc) + np.exp(-dE1*(tf-tc))) + ra11*np.exp(-dE1*tf))

    
    best={}
    for irow,ens in enumerate(yu.ensembles):
        t={
        'gS_j+':yu.ens2info[ens]['factor_gS'],
        'gS_j-':yu.ens2info[ens]['factor_gS'],
        'gA_j+':yu.ens2info[ens]['factor_gAs'],
        'gA_j-':yu.ens2info[ens]['factor_gAv'],
        'gT_j+':yu.ens2info[ens]['factor_gT'],
        'gT_j-':yu.ens2info[ens]['factor_gT'],
        }
        prefactor=t[ins] if ins in t else 1
        
        xUnit=yu.ens2info[ens]['a']
        
        for icol,case in enumerate(['disc']):
            icol*=3
            ylim=settings['ylim']
            shift_chi2=(ylim[1]-ylim[0])/10
            needsVEV=True if case!='conn' and ins in ['gS_j+'] else False
            tfs=[int(insert.split('_')[-1]) for insert in data[ens]['3pt']['t;0,0,0;0;N1_t;0,0,0;0;N1'].keys() if insert.startswith('id_j+')]; tfs.sort()
            tfs=[tf for tf in tfs if tf<=settings['tfmax_3pt'][irow]]
            
            oneQ = False
            dat=getdat(ens,ins,tfs,case)
            
            # ratio plot
            def func(dat):
                t=yu.meanDeep(dat)
                t=[t[0][i][1:tf]/np.sqrt(
                    np.real(t[1][0][tf])*np.real(t[1][1][tf])*\
                    np.real(t[1][0][1:tf][::-1])/np.real(t[1][0][1:tf])*\
                    np.real(t[1][1][1:tf])/np.real(t[1][1][1:tf][::-1])
                ) - (t[2][0] if needsVEV else 0) 
                for i,tf in enumerate(tfs)]
                # print(t)
                t=yu.prefactorDeep(t,prefactor)
                return t
            (mean,err,cov) = yu.jackknife(dat,func)
            for i_tf,tf in enumerate(tfs):
                tMean=mean[i_tf];tErr=err[i_tf]
                axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)*xUnit,tMean,tErr,fmt=fmts[irow])
                axs[irow,icol].set_ylim(ylim)
            # axs[irow,icol].set_xlim([-12*xUnit,12*xUnit])
                
            # mid point plot
            def func(dat):
                t=yu.meanDeep(dat)
                t=np.real([prefactor*np.array(t[0][i][1:tf]/np.sqrt(
                    np.real(t[1][0][tf])*np.real(t[1][1][tf])*\
                    np.real(t[1][0][1:tf][::-1])/np.real(t[1][0][1:tf])*\
                    np.real(t[1][1][1:tf])/np.real(t[1][1][1:tf][::-1])
                ) - (t[2][0] if needsVEV else 0))[tf//2]
                for i,tf in enumerate(tfs)])
                return t
            (mean,err,cov) = yu.jackknife(dat,func)
            tMean=mean[0];tErr=err[0]
            axs[irow,icol+1].errorbar(np.array(tfs)*xUnit,tMean,tErr,fmt=fmts[irow])
            axs[irow,icol+1].axis('auto')
            # axs[irow,icol+1].set_xlim([0,13*xUnit])
            
    #         fits=[]
    #         for datmin in settings['datmin'][irow]:
    #             print(ins,ens,case,datmin,end='                      \r')
    #             datmin_2pt=datmin_3pt_tf=datmin_3pt_tci=datmin_3pt_tcf=datmin
    #             datmin_2pt=ens2tfmin_best[ens]
    #             tfs_3pt_fit=[tf for tf in tfs if tf>=datmin_3pt_tf and tf+1>(datmin_3pt_tci+datmin_3pt_tcf)]
    #             tcs_3pt_fit={tf:np.arange(datmin_3pt_tci,tf+1-datmin_3pt_tcf) for tf in tfs_3pt_fit}
                
    #             def func(dat):
    #                 t=yu.meanDeep(dat)
    #                 t=np.real([prefactor*np.array(t[0][i][1:tf]/np.sqrt(
    #                     np.real(t[1][0][tf])*np.real(t[1][1][tf])*\
    #                     np.real(t[1][0][1:tf][::-1])/np.real(t[1][0][1:tf])*\
    #                     np.real(t[1][1][1:tf])/np.real(t[1][1][1:tf][::-1])
    #                 ) - (t[2][0] if needsVEV else 0))[tf//2]
    #                 for i,tf in enumerate(tfs) if tf in tfs_3pt_fit])
    #                 return t
                
    #             def fitfunc(g,c,dE):
    #                 return [g+c*np.exp(-dE*tf/2) for tf in tfs_3pt_fit]
                
    #             pars_mean,pars_err,pars_cov,chi2R_mean,chi2R_err,Ndof=yu.fit(dat,func,fitfunc,mask_cov=None,pars0=[-0.32716662, 0.73204631, 0.2])
    #             print(pars_mean,pars_err)
    #             fits.append((pars_mean,pars_err,chi2R_mean,Ndof))
    #             axs[irow,icol+2].errorbar([datmin],[pars_mean[0]],[pars_err[0]],fmt=fmts[irow],color='b',mfc='white')
    #             axs[irow,icol+2].annotate("%0.1f" %chi2R_mean,(datmin,pars_mean[0]-pars_err[0]-shift_chi2),color='b')
    #             axs[irow,icol+2].set_ylim(ylim)
                
    #         # modelAvg
    #         pars_mean_MA,pars_err_MA,probs=yu.modelAvg(fits)            
    #         t_mean=pars_mean_MA[0]; t_err=pars_err_MA[0]
    #         best[(ens,case)]=[yu.ens2info[ens]['a']**2,t_mean,t_err]
    #         axs[irow,icol+2].fill_between([0,settings['datmin'][irow][-1]+1],t_mean-t_err,t_mean+t_err,color='b',alpha=0.2)
    
    # for icol,case in [(2,'disc')]:
    #     # continue
    #     irow=len(yu.ensembles)
    #     a2s=np.array([best[(ens,case)][0] for ens in yu.ensembles])
    #     means=np.array([best[(ens,case)][1] for ens in yu.ensembles])
    #     errs=np.array([best[(ens,case)][2] for ens in yu.ensembles])
    #     axs[irow,icol].errorbar(a2s,means,errs,color='b',mfc='white')
    #     axs[irow,icol].set_ylim(ylim)
        
    #     def fitfunc(a2,g0):
    #         return g0+0*a2
    #     popt,pcov=curve_fit(fitfunc,a2s,means,sigma=errs,absolute_sigma=True)
    #     r=fitfunc(a2s,*popt)-means
    #     # print(r)
    #     # print(errs)
    #     chi2R=r.T @ np.linalg.inv(np.diag(errs**2)) @ r / (3-1)
    #     a2s2=np.arange(0,0.008,0.001)
    #     res=fitfunc(a2s2,*popt)
    #     axs[irow,icol].plot(a2s2,res,'r-')
    #     axs[irow,icol].fill_between(a2s2,res-[np.sqrt(pcov[0,0])]*len(a2s2),res+[np.sqrt(pcov[0,0])]*len(a2s2),color='r',alpha=0.2)
    #     axs[irow,icol].errorbar([0],[popt[0]],np.sqrt(pcov[0,0]),color='r')
    #     axs[irow,icol].annotate("%0.1f" %chi2R,(0,popt[0]-np.sqrt(pcov[0,0])-shift_chi2),color='r')
        
    #     def fitfunc(a2,g0,g1):
    #         return g0+g1*a2
    #     popt,pcov=curve_fit(fitfunc,a2s,means,sigma=errs,absolute_sigma=True)
    #     r=fitfunc(a2s,*popt)-means
    #     # print(r)
    #     # print(errs)
    #     chi2R=r.T @ np.linalg.inv(np.diag(errs**2)) @ r / (3-2)
    #     a2s2=np.arange(0,0.008,0.001)
    #     res=fitfunc(a2s2,*popt)
    #     es=np.array([np.sqrt(np.array([[1,a2]])@pcov@np.array([[1,a2]]).T)[0,0] for a2 in a2s2])
    #     axs[irow,icol].plot(a2s2,res,'b-')
    #     axs[irow,icol].fill_between(a2s2,res-es,res+es,color='b',alpha=0.2)
    #     axs[irow,icol].errorbar([0],[popt[0]],np.sqrt(pcov[0,0]),color='b')
    #     axs[irow,icol].annotate("%0.1f" %chi2R,(0,popt[0]-np.sqrt(pcov[0,0])-shift_chi2),color='b')

    plt.savefig('fig/'+ins+'_'+label+'.pdf')
    plt.close()
    
# settings={
#     'datmin':[range(1,8+1),range(1,9+1),range(1,9+1)],
#     'tfmax_3pt':[26,26,26],
#     'ylim':[-1,0.2],
# }
# run('gS_j-',settings)

# settings={
#     'datmin':[range(1,7+1),range(1,8+1),range(1,11+1)],
#     'tfmax_3pt':[20,22,26],
#     'ylim':[-0.03,0.03],
# }
# run('gA_j-',settings)

# settings={
#     'datmin':[range(1,4+1),range(1,4+1),range(1,4+1)],
#     'tfmax_3pt':[12,12,12],
#     'ylim':[-0.05,0.05],
# }
# run('gT_j-',settings)

In [ ]:
# 2st

def run(ins,settings,label='2st'):
    fig, axs = yu.getFigAxs(len(yu.ensembles)+1,2)
    fig.suptitle(ins)
    yu.addRowHeader(axs,[ens[:2]+ens[-2:] for ens in yu.ensembles]+['Extra'])
    yu.addColHeader(axs,['disc',''])
    
    for icol in [0,1]:
        axs[0,icol].get_shared_x_axes().join(axs[0,icol],*axs[1:len(yu.ensembles),icol])
    
    fmts=['s','s','s']
    
    func_C2pt_2st=lambda t,E0,c0,dE1,rc1: c0*np.exp(-E0*t)*(1 + rc1*np.exp(-dE1*t))
    func_C3pt_2st=lambda tf,tc,E0,a00,dE1,ra01,ra11: a00*np.exp(-E0*tf)*(1 + ra01*(np.exp(-dE1*tc) + np.exp(-dE1*(tf-tc))) + ra11*np.exp(-dE1*tf))

    
    best={}
    for irow,ens in enumerate(yu.ensembles):
        t={
        'gS_j+':yu.ens2info[ens]['factor_gS'],
        'gS_j-':yu.ens2info[ens]['factor_gS'],
        'gA_j+':yu.ens2info[ens]['factor_gAs'],
        'gA_j-':yu.ens2info[ens]['factor_gAv'],
        'gT_j+':yu.ens2info[ens]['factor_gT'],
        'gT_j-':yu.ens2info[ens]['factor_gT'],
        }
        prefactor=t[ins] if ins in t else 1
        
        xUnit=yu.ens2info[ens]['a']
        
        datmin_3pt_tf=settings['tfmin_3pt'][irow]
        
        for icol,case in enumerate(['disc']):
            icol*=3
            ylim=settings['ylim']
            shift_chi2=(ylim[1]-ylim[0])/10
            needsVEV=True if case!='conn' and ins in ['gS_j+'] else False
            tfs=[int(insert.split('_')[-1]) for insert in data[ens]['3pt']['t;0,0,0;0;N1_t;0,0,0;0;N1'].keys() if insert.startswith('id_j+')]; tfs.sort()
            tfs=[tf for tf in tfs if tf<=settings['tfmax_3pt'][irow] and tf>=datmin_3pt_tf]
            
            dat=getdat(ens,ins,tfs,case)
            
            # ratio plot
            def func(dat):
                t=yu.meanDeep(dat)
                t=[t[0][i][1:tf]/np.sqrt(
                    np.real(t[1][0][tf])*np.real(t[1][1][tf])*\
                    np.real(t[1][0][1:tf][::-1])/np.real(t[1][0][1:tf])*\
                    np.real(t[1][1][1:tf])/np.real(t[1][1][1:tf][::-1])
                ) - (t[2][0] if needsVEV else 0) 
                for i,tf in enumerate(tfs)]
                # print(t)
                t=yu.prefactorDeep(t,prefactor)
                return t
            (mean,err,cov) = yu.jackknife(dat,func)
            for i_tf,tf in enumerate(tfs):
                tMean=mean[i_tf];tErr=err[i_tf]
                axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)*xUnit,tMean,tErr,fmt=fmts[irow])
                axs[irow,icol].set_ylim(ylim)
            # axs[irow,icol].set_xlim([-12*xUnit,12*xUnit])
                
            # initial fits
            datmin=settings['datmin'][irow][0]
            datmin_3pt_tci=datmin_3pt_tcf=datmin
            datmin_2pt=8
            tfs_2pt_fit=np.arange(datmin_2pt,ens2tfmax[ens]+1)
            tfs_3pt_fit=[tf for tf in tfs if tf>=datmin_3pt_tf and tf+1>(datmin_3pt_tci+datmin_3pt_tcf)]
            tcs_3pt_fit={tf:np.arange(datmin_3pt_tci,tf+1-datmin_3pt_tcf) for tf in tfs_3pt_fit}
            
            pars0_initial=settings['pars_initial'][irow]
            E0,c0,E1=pars0_initial[:3]
            pars0=pars0_initial[-3:]
            def func(dat):
                t=yu.meanDeep(dat)
                t_3pt=[np.real(prefactor*(t[0][i]- (t[1][0][tf]*t[2][0] if needsVEV else 0)))[tcs_3pt_fit[tf]] for i,tf in enumerate(tfs) if tf in tfs_3pt_fit]
                return t_3pt
            def fitfunc(g,a01,a11):
                t_3pt=[func_C3pt_2st(tf,tcs_3pt_fit[tf],E0,g*c0,E1-E0,a01/(g*c0),a11/(g*c0)) for tf in tfs_3pt_fit]
                return t_3pt
            pars0_initial[-3:]=yu.fit(dat,func,fitfunc,pars0=pars0,jk=False)[0]
            
            pars0=pars0_initial
            # print(pars0)
            
            fits=[]
            for datmin in settings['datmin'][irow]:
                print(ins,ens,case,datmin,end='                      \r')
                datmin_2pt=datmin_3pt_tf=datmin_3pt_tci=datmin_3pt_tcf=datmin
                datmin_2pt=ens2tfmin_best[ens]
                tfs_2pt_fit=np.arange(datmin_2pt,ens2tfmax[ens]+1)
                tfs_3pt_fit=[tf for tf in tfs if tf>=datmin_3pt_tf and tf+1>(datmin_3pt_tci+datmin_3pt_tcf)]
                tcs_3pt_fit={tf:np.arange(datmin_3pt_tci,tf+1-datmin_3pt_tcf) for tf in tfs_3pt_fit}
                Ny_2pt=len(tfs_2pt_fit); Ny_3pt=len([1 for tf in tfs_3pt_fit for tc in tcs_3pt_fit[tf]])
                Ny=Ny_2pt+Ny_3pt
                mask_cov=np.array([[1 if (i<Ny_2pt and j<Ny_2pt) or (i>=Ny_2pt and j>=Ny_2pt) else 0 for j in range(Ny)] for i in range(Ny)])
                
                def func(dat):
                    t=yu.meanDeep(dat)
                    t_2pt=np.real(t[1][0][tfs_2pt_fit])
                    t_3pt=[np.real(prefactor*(t[0][i]- (t[1][0][tf]*t[2][0] if needsVEV else 0)))[tcs_3pt_fit[tf]] for i,tf in enumerate(tfs) if tf in tfs_3pt_fit]
                    return [t_2pt]+t_3pt
                def fitfunc(E0,c0,E1,rc1,g,a01,a11):
                    t_2pt=func_C2pt_2st(tfs_2pt_fit,E0,c0,E1-E0,rc1)
                    t_3pt=[func_C3pt_2st(tf,tcs_3pt_fit[tf],E0,g*c0,E1-E0,a01/(g*c0),a11/(g*c0)) for tf in tfs_3pt_fit]
                    return [t_2pt]+t_3pt
                pars_mean,pars_err,pars_cov,chi2R_mean,chi2R_err,Ndof=yu.fit(dat,func,fitfunc,mask_cov=None,pars0=pars0)
                # pars0[1:]=pars_mean[1:]
                fits.append((pars_mean,pars_err,chi2R_mean,Ndof))
                
                axs[irow,icol+1].errorbar([datmin*xUnit],[pars_mean[4]],[pars_err[4]],fmt=fmts[irow],color='b',mfc='white')
                axs[irow,icol+1].annotate("%0.1f" %chi2R_mean,(datmin*xUnit,pars_mean[4]-pars_err[4]-shift_chi2),color='b')
                axs[irow,icol+1].set_ylim(ylim)
                
            # modelAvg
            pars_mean_MA,pars_err_MA,probs=yu.modelAvg(fits)            
            t_mean=pars_mean_MA[4]; t_err=pars_err_MA[4]
            best[(ens,case)]=[yu.ens2info[ens]['a']**2,t_mean,t_err]
            axs[irow,icol+1].fill_between([0,(settings['datmin'][irow][-1]+1)*xUnit],t_mean-t_err,t_mean+t_err,color='b',alpha=0.2)
            # axs[irow,icol+1].set_xlim([0,10*xUnit])
    
    for icol,case in [(1,'disc')]:
        # continue
        irow=len(yu.ensembles)
        a2s=np.array([best[(ens,case)][0] for ens in yu.ensembles])
        means=np.array([best[(ens,case)][1] for ens in yu.ensembles])
        errs=np.array([best[(ens,case)][2] for ens in yu.ensembles])
        axs[irow,icol].errorbar(a2s,means,errs,color='b',mfc='white')
        axs[irow,icol].set_ylim(ylim)
        
        def fitfunc(a2,g0):
            return g0+0*a2
        popt,pcov=curve_fit(fitfunc,a2s,means,sigma=errs,absolute_sigma=True)
        r=fitfunc(a2s,*popt)-means
        # print(r)
        # print(errs)
        chi2R=r.T @ np.linalg.inv(np.diag(errs**2)) @ r / (3-1)
        a2s2=np.arange(0,0.008,0.001)
        res=fitfunc(a2s2,*popt)
        axs[irow,icol].plot(a2s2,res,'r-')
        axs[irow,icol].fill_between(a2s2,res-[np.sqrt(pcov[0,0])]*len(a2s2),res+[np.sqrt(pcov[0,0])]*len(a2s2),color='r',alpha=0.2)
        axs[irow,icol].errorbar([0],[popt[0]],np.sqrt(pcov[0,0]),color='r')
        axs[irow,icol].annotate("%0.1f" %chi2R,(0,popt[0]-np.sqrt(pcov[0,0])-shift_chi2),color='r')
        
        def fitfunc(a2,g0,g1):
            return g0+g1*a2
        popt,pcov=curve_fit(fitfunc,a2s,means,sigma=errs,absolute_sigma=True)
        r=fitfunc(a2s,*popt)-means
        # print(r)
        # print(errs)
        chi2R=r.T @ np.linalg.inv(np.diag(errs**2)) @ r / (3-2)
        a2s2=np.arange(0,0.008,0.001)
        res=fitfunc(a2s2,*popt)
        es=np.array([np.sqrt(np.array([[1,a2]])@pcov@np.array([[1,a2]]).T)[0,0] for a2 in a2s2])
        axs[irow,icol].plot(a2s2,res,'b-')
        axs[irow,icol].fill_between(a2s2,res-es,res+es,color='b',alpha=0.2)
        axs[irow,icol].errorbar([0],[popt[0]],np.sqrt(pcov[0,0]),color='b')
        axs[irow,icol].annotate("%0.1f" %chi2R,(0,popt[0]-np.sqrt(pcov[0,0])-shift_chi2),color='b')

    plt.savefig('fig/'+ins+'_'+label+'.pdf')
    plt.close()

settings={
    'datmin':[range(2,6+1),range(2,6+1),range(3,7+1)],
    'tfmin_3pt':[8,8,10],
    'tfmax_3pt':[18,18,18],
    'ylim':[-0.9,0.4],
    # E0,c0,E1,rc1,g,ra01,ra11
    'pars_initial':[[3.80305345e-01, 1.62458675e-08, 5.97738157e-01, 8.44149000e-01,-0.5,-1e-8,-1e-8],
                    [3.23009445e-01, 1.77755504e-09, 4.80835211e-01, 1.13744059e+00,-0.5,-1e-8,-1e-8],
                    [2.72641767e-01, 3.73256804e-10, 4.64117754e-01, 9.62179328e+00,-0.5,-1e-8,-1e-8]], 
}
run('gS_j-',settings)

# yu.flag_fit_cov2err=True   
# settings={
#     'datmin':[range(1,7+1),range(1,7+1),range(1,7+1)],
#     'tfmin_3pt':[6,6,6],
#     'tfmax_3pt':[18,18,18],
#     'ylim':[-1,0.2],
#     # E0,c0,E1,rc1,g,ra01,ra11
#     'pars_initial':[[3.80305345e-01, 1.62458675e-08, 5.97738157e-01, 8.44149000e-01,-0.5,-1e-8,-1e-8],
#                     [3.23009445e-01, 1.77755504e-09, 4.80835211e-01, 1.13744059e+00,-0.5,-1e-8,-1e-8],
#                     [2.72641767e-01, 3.73256804e-10, 4.64117754e-01, 9.62179328e+00,-0.5,-1e-8,-1e-8]],
# }
# run('gS_j-',settings,label='2st_uc')
# yu.flag_fit_cov2err=False
  
settings={
    'datmin':[range(4,7+1),range(5,8+1),range(6,10+1)],
    'tfmin_3pt':[8,8,10],
    'tfmax_3pt':[18,20,26],
    'ylim':[-0.03,0.03],
    # E0,c0,E1,rc1,g,ra01,ra11
    'pars_initial':[[3.80305345e-01, 1.62458675e-08, 5.97738157e-01, 8.44149000e-01,0.02,-1e-8,-1e-8],
                    [3.23009445e-01, 1.77755504e-09, 4.80835211e-01, 1.13744059e+00,0.02,-1e-8,-1e-8],
                    [2.72641767e-01, 3.73256804e-10, 4.64117754e-01, 9.62179328e+00,0.02,-1e-8,-1e-8]], 
}
run('gA_j-',settings)


In [ ]:
for tf in range(6,14,2):
    print([tf*yu.ens2info[ens]['a'] for ens in yu.ensembles])

In [ ]:
# slowly cut

def run(ins,settings,label='2st'):
    fig, axs = yu.getFigAxs(len(yu.ensembles)+1,2)
    fig.suptitle(ins)
    yu.addRowHeader(axs,[ens[:2]+ens[-2:] for ens in yu.ensembles]+['Extra'])
    yu.addColHeader(axs,['disc',''])
    
    for icol in [0,1]:
        axs[0,icol].get_shared_x_axes().join(axs[0,icol],*axs[1:len(yu.ensembles),icol])
    
    fmts=['s','s','s']
    
    func_C2pt_2st=lambda t,E0,c0,dE1,rc1: c0*np.exp(-E0*t)*(1 + rc1*np.exp(-dE1*t))
    func_C3pt_2st=lambda tf,tc,E0,a00,dE1,ra01,ra11: a00*np.exp(-E0*tf)*(1 + ra01*(np.exp(-dE1*tc) + np.exp(-dE1*(tf-tc))) + ra11*np.exp(-dE1*tf))

    
    best={}
    for irow,ens in enumerate(yu.ensembles):
        t={
        'gS_j+':yu.ens2info[ens]['factor_gS'],
        'gS_j-':yu.ens2info[ens]['factor_gS'],
        'gA_j+':yu.ens2info[ens]['factor_gAs'],
        'gA_j-':yu.ens2info[ens]['factor_gAv'],
        'gT_j+':yu.ens2info[ens]['factor_gT'],
        'gT_j-':yu.ens2info[ens]['factor_gT'],
        }
        prefactor=t[ins] if ins in t else 1
        
        
        xUnit=yu.ens2info[ens]['a']
        
        datmin_3pt_tf=settings['tfmin_3pt'][irow]
        
        for icol,case in enumerate(['disc']):
            icol*=3
            ylim=settings['ylim']
            shift_chi2=(ylim[1]-ylim[0])/10
            needsVEV=True if case!='conn' and ins in ['gS_j+'] else False
            tfs=[int(insert.split('_')[-1]) for insert in data[ens]['3pt']['t;0,0,0;0;N1_t;0,0,0;0;N1'].keys() if insert.startswith('id_j+')]; tfs.sort()
            tfs=[tf for tf in tfs if tf<=settings['tfmax_3pt'][irow] and tf>=datmin_3pt_tf]
            
            dat=getdat(ens,ins,tfs,case)
            
            # ratio plot
            def func(dat):
                t=yu.meanDeep(dat)
                t=[t[0][i][1:tf]/np.sqrt(
                    np.real(t[1][0][tf])*np.real(t[1][1][tf])*\
                    np.real(t[1][0][1:tf][::-1])/np.real(t[1][0][1:tf])*\
                    np.real(t[1][1][1:tf])/np.real(t[1][1][1:tf][::-1])
                ) - (t[2][0] if needsVEV else 0) 
                for i,tf in enumerate(tfs)]
                # print(t)
                t=yu.prefactorDeep(t,prefactor)
                return t
            (mean,err,cov) = yu.jackknife(dat,func)
            for i_tf,tf in enumerate(tfs):
                tMean=mean[i_tf];tErr=err[i_tf]
                axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)*xUnit,tMean,tErr,fmt=fmts[irow])
                axs[irow,icol].set_ylim(ylim)
            # axs[irow,icol].set_xlim([-12*xUnit,12*xUnit])
                
            # initial fits
            datmin=settings['datmin'][irow][0]
            datmin_3pt_tci=datmin_3pt_tcf=datmin
            datmin_2pt=ens2tfmin_best[ens]
            tfs_2pt_fit=np.arange(datmin_2pt,ens2tfmax[ens]+1)
            tfs_3pt_fit=[tf for tf in tfs if tf>=datmin_3pt_tf and tf+1>(datmin_3pt_tci+datmin_3pt_tcf)]
            tcs_3pt_fit={tf:np.arange(datmin_3pt_tci,tf+1-datmin_3pt_tcf) for tf in tfs_3pt_fit}
            
            pars0_initial=settings['pars_initial'][irow]
            E0,c0,E1=pars0_initial[:3]
            pars0=pars0_initial[-3:]
            def func(dat):
                t=yu.meanDeep(dat)
                t_3pt=[np.real(prefactor*(t[0][i]- (t[1][0][tf]*t[2][0] if needsVEV else 0)))[tcs_3pt_fit[tf]] for i,tf in enumerate(tfs) if tf in tfs_3pt_fit]
                return t_3pt
            def fitfunc(g,a01,a11):
                t_3pt=[func_C3pt_2st(tf,tcs_3pt_fit[tf],E0,g*c0,E1-E0,a01/(g*c0),a11/(g*c0)) for tf in tfs_3pt_fit]
                return t_3pt
            pars0_initial[-3:]=yu.fit(dat,func,fitfunc,pars0=pars0,jk=False)[0]
            pars0=pars0_initial
            # print(pars0)
            
            fits=[]
            for Ncut in range(0,100,2):
                print(ins,ens,case,Ncut,end='                      \r')
                
                if Ncut!=0:
                    tctfs=[tcs_3pt_fit[tf] for tf in tfs_3pt_fit]
                    for i in range(len(tctfs)):
                        if i==len(tctfs)-1:
                            tctfs[i]=tctfs[i][1:-1]
                            break
                        if tctfs[i][0]==tctfs[-1][0]:
                            tctfs[i]=tctfs[i][1:-1]
                            break
                    if len(tctfs[0])==0:
                        tctfs.pop(0)
                        tfs_3pt_fit.pop(0)
                    if len(tctfs)==0:
                        break
                    tcs_3pt_fit={tf:tctfs[i] for i,tf in enumerate(tfs_3pt_fit)}
                        
                # print(Ncut)
                # print([tcs_3pt_fit[tf] for tf in tfs_3pt_fit])
                # print()
                
                if Ncut not in settings['Ncuts'][irow]:
                    continue
                
                Ny_2pt=len(tfs_2pt_fit); Ny_3pt=len([1 for tf in tfs_3pt_fit for tc in tcs_3pt_fit[tf]])
                Ny=Ny_2pt+Ny_3pt
                mask_cov=np.array([[1 if (i<Ny_2pt and j<Ny_2pt) or (i>=Ny_2pt and j>=Ny_2pt) else 0 for j in range(Ny)] for i in range(Ny)])
            
                def func(dat):
                    t=yu.meanDeep(dat)
                    t_2pt=np.real(t[1][0][tfs_2pt_fit])
                    t_3pt=[np.real(prefactor*(t[0][i]- (t[1][0][tf]*t[2][0] if needsVEV else 0)))[tcs_3pt_fit[tf]] for i,tf in enumerate(tfs) if tf in tfs_3pt_fit]
                    return [t_2pt]+t_3pt
                def fitfunc(E0,c0,E1,rc1,g,a01,a11):
                    t_2pt=func_C2pt_2st(tfs_2pt_fit,E0,c0,E1-E0,rc1)
                    t_3pt=[func_C3pt_2st(tf,tcs_3pt_fit[tf],E0,g*c0,E1-E0,a01/(g*c0),a11/(g*c0)) for tf in tfs_3pt_fit]
                    return [t_2pt]+t_3pt
                pars_mean,pars_err,pars_cov,chi2R_mean,chi2R_err,Ndof=yu.fit(dat,func,fitfunc,mask_cov=None,pars0=pars0)
                # pars0[1:]=pars_mean[1:]
                fits.append((pars_mean,pars_err,chi2R_mean,Ndof))
                
                axs[irow,icol+1].errorbar([Ncut],[pars_mean[4]],[pars_err[4]],fmt=fmts[irow],color='b',mfc='white')
                axs[irow,icol+1].annotate("%0.1f" %chi2R_mean,(Ncut,pars_mean[4]-pars_err[4]-shift_chi2),color='b')
                axs[irow,icol+1].set_ylim(ylim)
                
            # modelAvg
            pars_mean_MA,pars_err_MA,probs=yu.modelAvg(fits)            
            t_mean=pars_mean_MA[4]; t_err=pars_err_MA[4]
            best[(ens,case)]=[yu.ens2info[ens]['a']**2,t_mean,t_err]
            axs[irow,icol+1].fill_between([0,settings['Ncuts'][irow][-1]],t_mean-t_err,t_mean+t_err,color='b',alpha=0.2)
    
    for icol,case in [(1,'disc')]:
        # continue
        irow=len(yu.ensembles)
        a2s=np.array([best[(ens,case)][0] for ens in yu.ensembles])
        means=np.array([best[(ens,case)][1] for ens in yu.ensembles])
        errs=np.array([best[(ens,case)][2] for ens in yu.ensembles])
        axs[irow,icol].errorbar(a2s,means,errs,color='b',mfc='white')
        axs[irow,icol].set_ylim(ylim)
        
        def fitfunc(a2,g0):
            return g0+0*a2
        popt,pcov=curve_fit(fitfunc,a2s,means,sigma=errs,absolute_sigma=True)
        r=fitfunc(a2s,*popt)-means
        # print(r)
        # print(errs)
        chi2R=r.T @ np.linalg.inv(np.diag(errs**2)) @ r / (3-1)
        a2s2=np.arange(0,0.008,0.001)
        res=fitfunc(a2s2,*popt)
        axs[irow,icol].plot(a2s2,res,'r-')
        axs[irow,icol].fill_between(a2s2,res-[np.sqrt(pcov[0,0])]*len(a2s2),res+[np.sqrt(pcov[0,0])]*len(a2s2),color='r',alpha=0.2)
        axs[irow,icol].errorbar([0],[popt[0]],np.sqrt(pcov[0,0]),color='r')
        axs[irow,icol].annotate("%0.1f" %chi2R,(0,popt[0]-np.sqrt(pcov[0,0])-shift_chi2),color='r')
        
        def fitfunc(a2,g0,g1):
            return g0+g1*a2
        popt,pcov=curve_fit(fitfunc,a2s,means,sigma=errs,absolute_sigma=True)
        r=fitfunc(a2s,*popt)-means
        # print(r)
        # print(errs)
        chi2R=r.T @ np.linalg.inv(np.diag(errs**2)) @ r / (3-2)
        a2s2=np.arange(0,0.008,0.001)
        res=fitfunc(a2s2,*popt)
        es=np.array([np.sqrt(np.array([[1,a2]])@pcov@np.array([[1,a2]]).T)[0,0] for a2 in a2s2])
        axs[irow,icol].plot(a2s2,res,'b-')
        axs[irow,icol].fill_between(a2s2,res-es,res+es,color='b',alpha=0.2)
        axs[irow,icol].errorbar([0],[popt[0]],np.sqrt(pcov[0,0]),color='b')
        axs[irow,icol].annotate("%0.1f" %chi2R,(0,popt[0]-np.sqrt(pcov[0,0])-shift_chi2),color='b')

    # plt.savefig('fig/'+ins+'_'+label+'.pdf')
    # plt.close()

# settings={
#     'datmin':[[1],[1],[1]],
#     'Ncuts':[range(0,60,2),range(0,60,2),range(0,60,2)],
#     'tfmin_3pt':[6,6,6],
#     'tfmax_3pt':[18,18,18],
#     'ylim':[-1,0.2],
#     # E0,c0,E1,rc1,g,ra01,ra11
#     'pars_initial':[[3.80305345e-01, 1.62458675e-08, 5.97738157e-01, 8.44149000e-01,-0.5,-1e-8,-1e-8],
#                     [3.23009445e-01, 1.77755504e-09, 4.80835211e-01, 1.13744059e+00,-0.5,-1e-8,-1e-8],
#                     [2.72641767e-01, 3.73256804e-10, 4.64117754e-01, 9.62179328e+00,-0.5,-1e-8,-1e-8]], 
# }
# run('gS_j-',settings)

# yu.flag_fit_cov2err=True   
# settings={
#     'datmin':[range(1,7+1),range(1,7+1),range(1,7+1)],
#     'tfmin_3pt':[6,6,6],
#     'tfmax_3pt':[18,18,18],
#     'ylim':[-1,0.2],
#     # E0,c0,E1,rc1,g,ra01,ra11
#     'pars_initial':[[3.80305345e-01, 1.62458675e-08, 5.97738157e-01, 8.44149000e-01,-0.5,-1e-8,-1e-8],
#                     [3.23009445e-01, 1.77755504e-09, 4.80835211e-01, 1.13744059e+00,-0.5,-1e-8,-1e-8],
#                     [2.72641767e-01, 3.73256804e-10, 4.64117754e-01, 9.62179328e+00,-0.5,-1e-8,-1e-8]],
# }
# run('gS_j-',settings,label='2st_uc')
# yu.flag_fit_cov2err=False
  
# settings={
#     'datmin':[range(1,6+1),range(1,7+1),range(1,8+1)],
#     'tfmin_3pt':[6,6,6],
#     'tfmax_3pt':[18,20,22],
#     'ylim':[-0.03,0.03],
#     # E0,c0,E1,rc1,g,ra01,ra11
#     'pars_initial':[[3.80305345e-01, 1.62458675e-08, 5.97738157e-01, 8.44149000e-01,0.02,-1e-8,-1e-8],
#                     [3.23009445e-01, 1.77755504e-09, 4.80835211e-01, 1.13744059e+00,0.02,-1e-8,-1e-8],
#                     [2.72641767e-01, 3.73256804e-10, 4.64117754e-01, 9.62179328e+00,0.02,-1e-8,-1e-8]], 
# }
# run('gA_j-',settings)
